In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras.layers import (Activation, AveragePooling1D, BatchNormalization,
                          Conv1D, Dense, GlobalAveragePooling1D, Input)
from keras.layers.core import Lambda
from keras.models import Model
from keras.optimizers import Nadam
from keras.regularizers import l2
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import generic_utils
from keras.utils import np_utils,plot_model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from scipy import stats

In [ ]:
kernel_posterior_fn = tfp.layers.default_mean_field_normal_fn(
      untransformed_scale_initializer=tf.compat.v1.initializers.random_normal(
          mean=0,
          stddev=1),)
kernel_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (x_train.shape[0] *1.0)
def abs_backend(inputs):
    return K.abs(inputs)

def expand_dim_backend(inputs):
    return K.expand_dims(inputs,1)

def sign_backend(inputs):
    return K.sign(inputs)

def pad_backend(inputs, in_channels, out_channels):
    pad_dim = (out_channels - in_channels)//2
    inputs = K.expand_dims(inputs)
    inputs = K.spatial_2d_padding(inputs,padding=((0,0),(pad_dim,pad_dim)))
    return K.squeeze(inputs,-1)

def residual_shrinkage_block(incoming, nb_blocks, out_channels, downsample=False,
                             downsample_strides=2):
    residual = incoming
    in_channels = incoming.get_shape().as_list()[-1]
    
    for _ in range(nb_blocks):
        
        identity = residual
        
        if not downsample:
            downsample_strides = 1
        
        residual = BatchNormalization()(residual)
        residual = Activation('relu')(residual)
        residual = Conv1D(out_channels, 3, strides=downsample_strides, 
                          padding='same', kernel_initializer='he_normal', 
                          kernel_regularizer=l2(1e-4))(residual)
        
        residual = BatchNormalization()(residual)
        residual = Activation('relu')(residual)
        residual = Conv1D(out_channels, 3, padding='same', kernel_initializer='he_normal', 
                          kernel_regularizer=l2(1e-4))(residual)
        
        residual_abs = Lambda(abs_backend)(residual)
        abs_mean = GlobalAveragePooling1D()(residual_abs)
        scales = Dense(out_channels, activation=None, kernel_initializer='he_normal', 
                       kernel_regularizer=l2(1e-4))(abs_mean)
        scales = BatchNormalization()(scales)
        scales = Activation('relu')(scales)
        scales = Dense(out_channels, activation='sigmoid', kernel_regularizer=l2(1e-4))(scales)
        scales = Lambda(expand_dim_backend)(scales)
        
        thres = keras.layers.multiply([abs_mean, scales])
        
        sub = keras.layers.subtract([residual_abs, thres])
        zeros = keras.layers.subtract([sub, sub])
        n_sub = keras.layers.maximum([sub, zeros])
        residual = keras.layers.multiply([Lambda(sign_backend)(residual), n_sub])
        
        if downsample_strides > 1:
            identity = AveragePooling1D(pool_size=1, strides=2)(identity)
            
        if in_channels != out_channels:

            identity = Lambda(pad_backend, arguments={'in_channels':in_channels,'out_channels':out_channels})(identity)



        residual = keras.layers.add([residual, identity])
    
    return residual
if __name__ == '__main__':
    
    inputs = 16
    outputs = 2

    x_input = Input(shape=(inputs,1))
    x = Conv1D(4,3,strides=2,padding='same')(x_input)
    x = Conv1D(4,3,2,padding='same')(x)
    x = residual_shrinkage_block(x, 1, 4, downsample=False)
    x = residual_shrinkage_block(x, 3, 4, downsample=False)

    x = residual_shrinkage_block(x, 1, 8, downsample=True)

    x = BatchNormalization()(x)
    x = Activation('relu')(x)    
    x = GlobalAveragePooling1D()(x)

    x = Dense(outputs,activation='sigmoid')(x)
    
    model = Model(inputs=x_input,outputs=x)
    optimizers = Nadam(lr=1e-5)
    model.compile(optimizer = optimizers, loss= 'categorical_crossentropy',metrics=['accuracy'])
    model.summary()
    
    
    model.fit(x_train, y_train, validation_data=(x_test,y_test),epochs=100, batch_size=512, verbose=1)
    pred_vi=np.zeros((len(x_val),4))
    pred_max_p_vi=np.zeros((len(x_val)))
    pred_std_vi=np.zeros((len(x_val)))
    entropyaleatoric_vi = np.zeros((len(x_val)))
    entropyepistemic_vi = np.zeros((len(x_val)))
    for i in tqdm(range(0,len(x_val))):

        multi_img=np.tile(x_val[i],(5,1,1))
        preds=model.predict(multi_img)
        pred_vi[i]=np.mean(preds,axis=0)#mean over n runs of every proba class
        for j in range(0,4):
            for z in range (0,5):
                entropyaleatoric_vi[i] = -((np.sum(preds[z][j] * np.log2(preds[z][j] + 1E-14)))/5) #Numerical Stability
                entropyepistemic_vi[i] = (-np.sum(pred_vi[i][j] * np.log2(pred_vi[i][j] + 1E-14)))+((np.sum(preds[z][j] * np.log2(preds[z][j] + 1E-14)))/5)